In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)   

data = pd.read_csv('DATA_KOTA.csv')
data_kecelakaan = data['jumlah_kecelakaan']
X = data_kecelakaan[:-1].values.reshape(-1,1)
y = data_kecelakaan[1:].values.reshape(-1,1).ravel()

df = pd.DataFrame(data)

# Pastikan jumlah sampel dan target sama
assert len(X) == len(y), "Jumlah sampel dan target harus sama"

# Membagi kolom tanggal menjadi tahun dan bulan
df['tahun'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[0]))
df['bulan'] = df['tahun_bulan'].apply(lambda x: int(x.split('-')[1]))

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Memisahkan fitur dan target
X = df[['tahun', 'bulan']].values
y = df['jumlah_kecelakaan'].values

In [3]:
# Pisahkan data menjadi training dan testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [4]:
# Fungsi untuk menghitung MAPE menggunakan model SVR dengan parameter tertentu
def calculate_mape(X_train, y_train, X_val, y_val, params):
    C, epsilon, gamma = params
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_val)
    mape = mean_absolute_percentage_error(y_val, y_pred) *100
    return mape

# Inisialisasi populasi awal
def initialize_population(population_size, parameter_ranges):
    population = []
    for _ in range(population_size):
        chromosome = []
        for param_range in parameter_ranges:
            param_value = np.random.uniform(param_range[0], param_range[1])
            chromosome.append(param_value)
        population.append(chromosome)
    return population

# Seleksi individu berdasarkan nilai fitness (MAPE)
def selection(population, X_train, y_train, X_val, y_val):
    selected_population = []
    for chromosome in population:
        mape = calculate_mape(X_train, y_train, X_val, y_val, chromosome)
        selected_population.append((chromosome, mape))
    selected_population.sort(key=lambda x: x[1])  # Sort berdasarkan MAPE terendah
    return selected_population[:len(population)//2]  # Pilih setengah individu terbaik

# Crossover: Menghasilkan keturunan dari pasangan individu terpilih
def crossover(selected_population):
    new_population = []
    ukuran_populasi = len(selected_population)
    
    for i in range(ukuran_populasi):
        parent1 = selected_population[i][0]
        
        # Tentukan parent2 dengan pemeriksaan batas
        if i % 2 == 0:
            if i - 1 >= 0:
                parent2 = selected_population[i-1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i-1 tidak valid
        else:
            if i + 1 < ukuran_populasi:
                parent2 = selected_population[i+1][0]
            else:
                parent2 = selected_population[i][0]  # Penanganan alternatif jika i+1 tidak valid
        
        crossover_point = np.random.randint(1, len(parent1)-1)
        child = parent1[:crossover_point] + parent2[crossover_point:]
        new_population.append(child)
    
    return new_population

# Mutasi: Mengubah nilai gen secara acak pada populasi baru
def mutation(new_population, mutation_rate, parameter_ranges, sigma_range):
    for i in range(len(new_population)):
        if np.random.random() < mutation_rate:
            mutated_gene_index = np.random.randint(0, len(new_population[i]))
            sigma = np.random.uniform(sigma_range[0], sigma_range[1])
            new_value = new_population[i][mutated_gene_index] + np.random.normal(0, sigma)
            new_population[i][mutated_gene_index] = np.clip(new_value, parameter_ranges[mutated_gene_index][0], parameter_ranges[mutated_gene_index][1])
    return new_population

In [5]:
# Rentang nilai parameter optimal
parameter_ranges = [(0.0001, 0.1), (0.001, 1), (1e-07, 0.01)]

# Inisialisasi populasi awal
population_size = 70
population = initialize_population(population_size, parameter_ranges)

# Iterasi algoritma genetika
crossover_rate = 1  # Tetapkan tingkat crossover tetap untuk eksperimen
mutation_rate = 0.0  # Tetapkan tingkat mutasi tetap untuk eksperimen
num_generations = 250  # Tetapkan jumlah generasi tetap untuk eksperimen
sigma_range = (0.005, 0.01)

for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    selected_population = selection(population, X_train, y_train, X_val, y_val)
    for i, (chromosome, mape) in enumerate(selected_population):
        print(f"Individu {i + 1}: C={chromosome[0]}, epsilon={chromosome[1]}, gamma={chromosome[2]}, MAPE={mape}")
    
    new_population = crossover(selected_population)
    new_population = mutation(new_population, mutation_rate, parameter_ranges, sigma_range)

    # Adjust the population size if necessary
    if len(new_population) < population_size:
        additional_individuals = initialize_population(population_size - len(new_population), parameter_ranges)
        new_population.extend(additional_individuals)
    elif len(new_population) > population_size:
        new_population = new_population[:population_size]
    
    # Make sure the new population has the correct size
    assert len(new_population) == population_size, "Ukuran populasi baru tidak sesuai"
    
    population = new_population

# Mengambil individu terbaik (dengan MAPE terendah) dari populasi terakhir
best_chromosome, best_mape = min(selected_population, key=lambda x: x[1])

# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='rbf', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nParameter terbaik:")
print(f"C: {best_chromosome[0]}")
print(f"Epsilon: {best_chromosome[1]}")
print(f"Gamma: {best_chromosome[2]}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu 1: C=0.07083645052182495, epsilon=0.021563909801506645, gamma=0.009699101530634727, MAPE=37.59260428537432
Individu 2: C=0.05471635690639364, epsilon=0.18566960107000152, gamma=0.00969584931918281, MAPE=39.470386097628584
Individu 3: C=0.08774620140276, epsilon=0.25868368608744047, gamma=0.006599874461937188, MAPE=39.628346308815466
Individu 4: C=0.03417252846992083, epsilon=0.11436004771934848, gamma=0.0092469437134238, MAPE=39.71891825411668
Individu 5: C=0.03591072628157284, epsilon=0.11675319046560459, gamma=0.008631047948413347, MAPE=39.837996115892764
Individu 6: C=0.06455276176190404, epsilon=0.17519206257598646, gamma=0.00690940828725085, MAPE=39.85981170281178
Individu 7: C=0.044071234124586176, epsilon=0.12291619660993405, gamma=0.004951819583421692, MAPE=40.62086491294698
Individu 8: C=0.030530915540419732, epsilon=0.09857444189237749, gamma=0.006842361841818918, MAPE=41.02321841691081
Individu 9: C=0.024261043860955123, epsilon=0.09400966503809331, gamm

In [6]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[6.41679434]
 [6.495715  ]
 [8.49637389]
 [8.59912639]
 [7.15045591]
 [9.8533695 ]
 [7.82585266]
 [9.19094937]
 [7.06878983]
 [9.3418141 ]
 [8.40116331]
 [9.63858343]]


In [7]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='poly', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [-615979.07661565 -388429.37046941 -554755.05903753 -141516.75001653
 -550306.96013005 -259763.25984319 -162350.27037175 -534306.66774114
 -595591.25215276 -120662.85057195  -94924.43657048 -443334.03273137]
MAPE: 101156306.12804788


In [8]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[-13551539.6855443 ]
 [ -8545446.15032702]
 [-12204611.29882555]
 [ -3113368.50036364]
 [-12106753.1228612 ]
 [ -5714791.71655016]
 [ -3571705.94817858]
 [-11754746.69030505]
 [-13103007.54736071]
 [ -2654582.71258288]
 [ -2088337.60455065]
 [ -9753348.7200902 ]]


In [9]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='linear', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.15582446 0.201279   0.40781538 0.48963356 0.24891234 0.54635781
 0.40563659 0.49181235 0.23982143 0.57363053 0.49872447 0.50999417]
MAPE: 28.477978656679998


In [10]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[ 3.42813812]
 [ 4.4281381 ]
 [ 8.97193835]
 [10.77193834]
 [ 5.47607152]
 [12.01987175]
 [ 8.92400493]
 [10.81987176]
 [ 5.27607153]
 [12.61987177]
 [10.97193834]
 [11.21987175]]


In [11]:
# Menguji model regresi dengan parameter terbaik
best_svr = SVR(kernel='sigmoid', C=best_chromosome[0], epsilon=best_chromosome[1], gamma=best_chromosome[2])
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred) *100

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)


Hasil peramalan:
Prediksi: [0.34090909 0.34090909 0.34090909 0.34090909 0.34090909 0.34090909
 0.34090909 0.34090909 0.34090909 0.34090909 0.34090909 0.34090909]
MAPE: 43.84695165945167


In [12]:
pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1))
print(pred_denormalized)

[[7.5]
 [7.5]
 [7.5]
 [7.5]
 [7.5]
 [7.5]
 [7.5]
 [7.5]
 [7.5]
 [7.5]
 [7.5]
 [7.5]]
